# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [4]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [7]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2025/11/11/

In [8]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [11]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 7 relevant links


{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'curriculum / resume',
   'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter/X profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [10]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [12]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 2 relevant links


{'links': [{'type': 'company page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'}]}

In [ ]:
select_relevant_links("https://huggingface.co")

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [13]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [14]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 8 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
tencent/HY-MT1.5-1.8B
Updated
6 days ago
•
6.61k
•
636
Qwen/Qwen-Image-2512
Updated
7 days ago
•
16.8k
•
503
LGAI-EXAONE/K-EXAONE-236B-A23B
Updated
1 day ago
•
2.67k
•
416
Lightricks/LTX-2
Updated
6 minutes ago
•
84.4k
•
352
IQuestLab/IQuest-Coder-V1-40B-Loop-Instruct
Updated
4 days ago
•
6.32k
•
264
Browse 2M+ models
Spaces
Running
Featured
3.83k
Wan2.2 Animate
👁
3.83k
Wan2.2 Animate
Running
on
Zero
1.09k
Z Image Turbo
🖼
1.09k
Generate images from text prompts
Running
on
Zero
MCP
Featured
240
Qwen-Image-Edit-2511-LoRAs-Fast
🎃
240
Dem

In [15]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\ntencent/HY-MT1.5-1.8B\nUpdated\n6 days ago\n•\n6.61k\n•\n636\nQwen/Qwen-Image-2512\nUpdated\n7 days ago\n•\n16.8k\n•\n503\nLGAI-EXAONE/K-EXAONE-236B-A23B\nUpdated\n1 day ago\n•\n2.67k\n•\n416\nLightricks/LTX-2\nUpdated\n7 minutes ago\n•\n84.4k\n•\n352\nIQuestLab/IQuest-Coder-V1-40B-Loop-Instruct\nUpdated\n4 days ago\n•\n6.32k\n•\n264\nBrowse 2M+ models\nSpaces\nRunning\nFeatured\n3.83k\nWan

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 16 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the AI community building the future of machine learning (ML). It is a vibrant collaboration platform that serves as a central hub for the ML community worldwide. Hugging Face empowers machine learning engineers, scientists, and end users to discover, share, and experiment with open-source ML models, datasets, and applications — spanning modalities such as text, image, video, audio, and even 3D.

With over 2 million models and 500,000 datasets available, Hugging Face is the home of machine learning for collaboration, discovery, and accelerating innovation in AI.

---

## What We Offer

### The Hugging Face Hub

- **Models:** Browse and collaborate on over 2 million ML models that cover a wide range of AI applications.
- **Datasets:** Access and contribute to more than 500,000 datasets for training and experimentation.
- **Spaces:** Run and share AI-powered applications, including demos and interactive apps.
- **Applications:** Discover over 1 million AI-powered applications created by the community.

### Platform Features

- **Open Source Stack:** Leverage state-of-the-art open-source tools and infrastructure for fast experimentation and deployment.
- **Multi-modality Support:** Explore AI models and datasets across text, images, videos, audio, and 3D data.
- **Collaboration:** Host and collaborate on unlimited public models, datasets, and AI apps to build your ML portfolio.
- **Compute & Enterprise Solutions:** Access scalable paid compute resources tailored for teams and enterprises.

---

## Company Culture

Hugging Face is deeply rooted in open and ethical AI development with a strong focus on community collaboration. The company believes in:

- **Transparency:** Promoting open access to models and datasets.
- **Community Empowerment:** Supporting and encouraging contributions from the next generation of ML engineers and scientists.
- **Ethical AI:** Building tools and resources that lead to responsible development and deployment of AI technologies.

---

## Our Customers & Community

Our user base spans individual ML researchers, AI developers, startups, academic institutions, and established enterprises looking to leverage cutting-edge AI technology.

Top organizations and communities use Hugging Face to accelerate their AI projects by tapping into the rich repository of open-source models and datasets coupled with a collaborative environment.

---

## Careers at Hugging Face

Joining Hugging Face means working at the forefront of AI innovation alongside a passionate, diverse team dedicated to making machine learning accessible and ethical.

- **Opportunities:** Roles for engineers, researchers, community managers, and enterprise solution specialists.
- **Environment:** A collaborative, fast-paced workplace that values contribution, learning, and impact.
- **Growth:** Build your career by working on groundbreaking AI technology with a global user base.

Explore opportunities to be part of a company that is not only shaping technology but building a responsible AI future.

---

## Connect with Us

- **Sign Up & Join the Community:** Build your ML portfolio and share your innovative work.
- **Explore Models & Datasets:** Dive into millions of open resources to enhance your AI projects.
- **Enterprise Solutions:** Bring AI to your team or company with advanced platform offerings.

Visit [huggingface.co](https://huggingface.co) to learn more and start collaborating today.

---

## Brand Colors & Logo

- Primary Colors:  
  - Yellow: #FFD21E  
  - Orange: #FF9D00  
  - Gray: #6B7280

Hugging Face logos and brand assets are available in .svg, .png, and .ai formats.

---

Hugging Face — *The AI community building the future.*  
Empower your machine learning journey with us.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 7 relevant links


# Hugging Face Brochure

---

## Who We Are  
**Hugging Face** is the global AI community and collaboration platform dedicated to building the future of machine learning. At our core, we empower researchers, developers, and organizations to create, share, and advance machine learning models, datasets, and applications.  

Our mission is to build an open and ethical AI future by fostering an inclusive community where innovation and openness thrive. We support the next generation of machine learning professionals by providing tools, libraries, and resources that accelerate research and deployment.

---

## What We Offer  

### Collaboration & Community  
- A vibrant and fast-growing community where ML engineers, scientists, and enthusiasts come together.  
- Hosting for **2 million+ ML models**, **500k+ datasets**, and **1 million+ applications**, spanning modalities such as text, image, video, audio, and even 3D.  
- Open-source hub to share and discover high-quality machine learning assets.  

### Tools & Resources  
- **Hugging Face Hub:** Centralized platform for sharing and experimenting with machine learning projects.  
- **Spaces:** Run and showcase applications easily on the platform with various computational backends.  
- Comprehensive documentation, forums, and tutorials to support beginners and experts alike.

### Enterprise & Compute Solutions  
- Paid Compute and Enterprise plans for teams and businesses seeking scalable, secure, and advanced AI infrastructure and support.  
- Tailored platforms to accelerate in-house AI development and integration.

---

## Customers & Use Cases  
Hugging Face supports a broad spectrum of users from individuals to large enterprises, powering innovations across diverse industries:  
- **Technology & Startups** leveraging our models to build next-gen AI products.  
- **Research institutions** collaborating on open scientific advancements.  
- **Enterprises** seeking scalable AI operations and deployment.  
- **Creative industries** using machine learning for image, video, and text understanding and generation.  

Our community-driven approach ensures cutting-edge solutions and rapid iteration cycles, with many popular open-source ML libraries originating from and maintained through Hugging Face.

---

## Company Culture  
- **Openness & Collaboration:** We thrive on open-source principles and community involvement to democratize AI innovation.  
- **Ethical AI:** Committed to advancing responsible AI development that benefits humanity as a whole.  
- **Diversity & Inclusion:** A vibrant, global community that welcomes talent from all backgrounds and expertise levels.  
- **Innovation:** Driven by curiosity and scientific rigor, continuously pushing the boundaries of machine learning technology.

---

## Careers at Hugging Face  
Join a passionate team at the forefront of the AI revolution! We offer:  
- Opportunities to work with cutting-edge technology and an active open-source ecosystem.  
- Roles for engineers, researchers, product managers, and community builders.  
- A dynamic, remote-friendly work environment promoting growth, creativity, and impact.  

Explore current openings and learn more about our recruitment process at our [Careers Page](https://huggingface.co/careers).

---

## Get Involved & Connect  
- Visit us at **huggingface.co** to explore models, datasets, or start building your own ML projects.  
- Join our community on Github, Twitter, Discord, and LinkedIn to stay updated and participate in conversations.  
- Access comprehensive documentation and tutorials directly on our website.  

---

## Brand Identity  
- Recognized by our vibrant yellow (#FFD21E) and an accessible visual identity promoting friendliness and approachability.  
- Our logo and brand assets are freely available to represent our open and welcoming community culture.  

---

Hugging Face: **The AI community building the future.**  
Join us to create, share, and shape the future of machine learning together!  

---

*For more information, visit:*  
[https://huggingface.co](https://huggingface.co)  
[GitHub](https://github.com/huggingface) | [Twitter](https://twitter.com/huggingface) | [LinkedIn](https://linkedin.com/company/huggingface) | [Discord](https://discord.gg/huggingface)

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>